# New York City Taxi Cab Trip

We look at [the New York City Taxi Cab dataset](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). This includes every ride made in the city of New York since 2009.

On [this website](http://chriswhong.github.io/nyctaxi/) you can see the data for one random NYC yellow taxi on a single day.

On [this post](http://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/), you can see an analysis of this dataset. Postgres and R scripts are available on [GitHub](https://github.com/toddwschneider/nyc-taxi-data).

## Loading the data

Normally we would read and load this data into memory as a Pandas dataframe.  However in this case that would be unwise because this data is too large to fit in RAM.

The data can stay in the hdfs filesystem but for performance reason we can't use the csv format. The file is large (32Go) and text formatted. Data Access is very slow.

## Parquet file format

[Parquet format](https://github.com/apache/parquet-format) is a common binary data store, used particularly in the Hadoop/big-data sphere. It provides several advantages relevant to big-data processing:

- columnar storage, only read the data of interest
- efficient binary packing
- choice of compression algorithms and encoding
- split data into files, allowing for parallel processing
- range of logical types
- statistics stored in metadata allow for skipping unneeded chunks
- data partitioning using the directory structure

To convert the csv file to parquet we can use Spark.
```python
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master('local[4]') \
        .config('spark.hadoop.parquet.enable.summary-metadata', 'true') \
        .getOrCreate()
df = spark.read.csv(
"hdfs://svmass2.mass.uhb.fr:54310/user/datasets/nyc-tlc/2009/yellow_tripdata_2009-01.csv", 
                    header="true",inferSchema="true")
df.write.parquet("hdfs://svmass2.mass.uhb.fr:54310/user/navaro_p/nyc-taxi/2019-01.parquet")
spark.stop()
```

To read multiple files
```py
spark.read.format("csv").option("header", "true").load("../Downloads/*.csv")
```

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master('spark://svmass2.mass.uhb.fr:7077') \
        .config('spark.hadoop.parquet.enable.summary-metadata', 'true') \
        .config("spark.executor.instances", 4) \
        .getOrCreate()
df = spark.read.csv(
    "hdfs://svmass2.mass.uhb.fr:54310/user/datasets/nyc-tlc/2009/yellow_tripdata_2009-01.csv", 
                    header="true",inferSchema="true")
df.write.parquet('hdfs://svmass2.mass.uhb.fr:54310/user/navaro_p/nyc-taxi/2019-01.parquet')
spark.stop()

KeyboardInterrupt: 

## Read  `parquet` file with Python

[fastparquet](http://fastparquet.readthedocs.io/en/latest/) provides a performant library to read and write Parquet files from Python, without any need for a Python-Java bridge. This will make the Parquet format an ideal storage mechanism for Python-based big data workflows.

The tabular nature of Parquet is a good fit for the Pandas data-frame objects, and we exclusively deal with data-frame <-> Parquet.

We will use a software layer to access the data. For performance reason the data must be aligned in memory along columns. It is not the default in Python. Apache Arrow will do this for you and improve performance.

## Apache Arrow

[Arrow](https://arrow.apache.org/docs/python/) is a columnar in-memory analytics layer designed to accelerate big data. It houses a set of canonical in-memory representations of flat and hierarchical data along with multiple language-bindings for structure manipulation.

https://arrow.apache.org/docs/python/parquet.html

The Apache Parquet project provides a standardized open-source columnar storage format for use in data analysis systems. It was created originally for use in Apache Hadoop with systems like Apache Drill, Apache Hive, Apache Impala, and Apache Spark adopting it as a shared standard for high performance data IO.

Apache Arrow is an ideal in-memory transport layer for data that is being read or written with Parquet files. [PyArrow](https://arrow.apache.org/docs/python/) includes Python bindings to read and write Parquet files with pandas.

For lighter data, it is recommanded to use [Feather](https://github.com/wesm/feather). It is a fast, interoperable data frame storage that comes with bindings for python and R.

Feather uses also the Apache Arrow columnar memory specification to represent binary data on disk. This makes read and write operations very fast.

In [2]:
%env HADOOP_HOME=/export/hadoop-2.7.6

import pyarrow as pa
import pyarrow.parquet as pq

hdfs = pa.hdfs.connect('svmass2.mass.uhb.fr', 54311, 'navaro_p')

env: HADOOP_HOME=/export/hadoop-2.7.6


ArrowIOError: Unable to load libhdfs

In [ ]:
import fastparquet as fp
pf = fp.ParquetFile('/user/navaro_p/2016-yellow.parquet', open_with=hdfs.open)
pf

In [5]:
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.master('spark://schedulers:7077').getOrCreate()
#spark

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master('spark://hadoop1:7077') \
        .getOrCreate()
spark

In [7]:
columns = ['tpep_pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'tip_amount', 'total_amount']

df = (spark.read.parquet('hdfs://hadoop1:9000/user/navaro/nyc-taxi/2016.parquet')
           .select(*columns))

In [8]:
# Sum the total number of passengers
df.agg({'passenger_count': 'sum'}).collect()

[Row(sum(passenger_count)=217355302)]

In [9]:
# Average number of passenger per trip.
df.agg({'passenger_count': 'avg'}).collect()

[Row(avg(passenger_count)=1.6571130312517794)]

In [10]:
# How many trip with 0,1,2,3,...,9 passengers
df.groupby('passenger_count').agg({'*': 'count'}).collect()

[Row(passenger_count=1, count(1)=92987719),
 Row(passenger_count=6, count(1)=4234423),
 Row(passenger_count=3, count(1)=5456807),
 Row(passenger_count=5, count(1)=6773026),
 Row(passenger_count=9, count(1)=261),
 Row(passenger_count=4, count(1)=2660369),
 Row(passenger_count=8, count(1)=316),
 Row(passenger_count=7, count(1)=361),
 Row(passenger_count=2, count(1)=19038307),
 Row(passenger_count=0, count(1)=13454)]

## Exercise
How well people tip based on the number of passengers in a cab.  To do this you have to:

1.  Remove rides with zero fare
2.  Add a new column `tip_fraction` that is equal to the ratio of the tip to the fare
3.  Group by the `passenger_count` column and take the mean of the `tip_fraction` column.

### Cheat Sheets and documentation
- [Spark DataFrames in Python](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf)
- [Spark in Python](http://datacamp-community.s3.amazonaws.com/4d91fcbc-820d-4ae2-891b-f7a436ebefd4)
-  https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Refer to the [Spark UI](../../../9070) for feedback.

## Hints

### How to remove rows

In Spark you can filter rows by a boolean expression like the following:

```python
df = df.filter(df.name == 'expression')
```

### How to make new columns

In Pandas you can create a new column using Python's setitem syntax like the following:

```python
df = df.withColumn('var2', df.var0 + df.var1)
```

### How to do groupby-aggregations

In Pandas you can do a groupby-aggregation by using the `groupby` method, followed by a column name an aggregation method like the following:

```python
df.groupBy(df.name).agg({'column-name': 'avg'})
```

When you want to collect the result of your computation, finish with the `.collect()` method.

In [11]:
## Your code here...

[Row(passenger_count=1, avg(tip_fraction)=0.17575518500251763),
 Row(passenger_count=6, avg(tip_fraction)=0.13983510145473738),
 Row(passenger_count=3, avg(tip_fraction)=0.14531817942803918),
 Row(passenger_count=5, avg(tip_fraction)=0.14320540950610772),
 Row(passenger_count=9, avg(tip_fraction)=1.5275117397957647),
 Row(passenger_count=4, avg(tip_fraction)=0.12744269701699368),
 Row(passenger_count=8, avg(tip_fraction)=0.29431817121207565),
 Row(passenger_count=7, avg(tip_fraction)=0.1641076331382901),
 Row(passenger_count=2, avg(tip_fraction)=0.15518521844025632),
 Row(passenger_count=0, avg(tip_fraction)=1.5782467400835083)]

In [12]:
spark.stop()

### Exercices 

1. Plot the tip as a function of the hour of day and the day of the week?
2. Investigate the `payment_type` column.  See how well each of the payment types correlate with the `tip_fraction`.  Did you find anything interesting?  Any guesses on what the different payment types might be?  If you're interested you may be able to find more information on the [NYC TLC's website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)
3.  How quickly can you get a taxi cab for a particular day of the year?  How about for a particular hour of that day?

